_**PART 2: SQLite**_
We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
 
QUESTIONS:

_**Q10**_: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

_**Q11**_: Produce a report of members and who recommended them in alphabetic surname,firstname order 


_**Q12**_: Find the facilities with their usage by member, but not guests

_**Q13**_: Find the facilities usage by month, but not guests



In [1]:
import sqlite3
from sqlite3 import Error


def create_connection(db_path):
    """
    Create a connection to an SQLite database.
    :param db_path: Path to the database file.
    :return: Connection object or None.
    """
    try:
        conn = sqlite3.connect(db_path)
        print(f"Connected to SQLite, version {sqlite3.version}")
        return conn
    except Error as e:
        print(f"Connection failed: {e}")
        return None


def fetch_all_facilities(conn):
    """
    Retrieve and print all records from the FACILITIES table.
    :param conn: SQLite Connection object.
    """
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM FACILITIES")
        rows = cursor.fetchall()

        if not rows:
            print("No records found in FACILITIES.")
        else:
            for row in rows:
                print(row)
    except Error as e:
        print(f"Error reading data: {e}")


def main():
    database = "sqlite_db_pythonsqlite.db"

    conn = create_connection(database)
    if conn:
        with conn:
            print("Querying all records from FACILITIES table:")
            fetch_all_facilities(conn)
    else:
        print("Database connection could not be established.")


if __name__ == "__main__":
    main()


Connected to SQLite, version 2.6.0
Querying all records from FACILITIES table:
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


C:\Users\maxmi\AppData\Local\Temp\ipykernel_26260\1958532945.py:13: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(f"Connected to SQLite, version {sqlite3.version}")


In [3]:
import sqlite3 
from sqlite3 import Error 
import pandas as pd
conn = sqlite3.connect("./sqlite_db_pythonsqlite.db")
cur = conn.cursor()
 

In [4]:
query = 'SELECT * FROM Facilities;'

In [6]:
df = pd.read_sql_query(query,conn)

In [7]:
print(df)

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


_**Q10**_: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [8]:
query_q10 = '''
SELECT f.name AS facility_name, 
       SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS total_revenue
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;
'''

In [10]:
df_q10 = pd.read_sql_query(query_q10, conn)
print("Facilities with total revenue less than 1000:")
print(df_q10)

Facilities with total revenue less than 1000:
   facility_name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


_**Q11**_: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [12]:
query_q11 = '''
SELECT m.firstname, m.surname, m.recommendedby
FROM Members m
ORDER BY m.surname, m.firstname;
'''

In [13]:
df_q11 = pd.read_sql_query(query_q11, conn)
print("\nMembers and who recommended them:")
print(df_q11)


Members and who recommended them:
    firstname            surname recommendedby
0    Florence              Bader             9
1        Anne              Baker             9
2     Timothy              Baker            13
3         Tim             Boothe             3
4      Gerald            Butters             1
5        Joan             Coplin            16
6       Erica            Crumpet             2
7       Nancy               Dare             4
8       David            Farrell              
9      Jemima            Farrell              
10      GUEST              GUEST              
11    Matthew            Genting             5
12       John               Hunt            30
13      David              Jones             4
14    Douglas              Jones            11
15     Janice           Joplette             1
16       Anna          Mackenzie             1
17    Charles               Owen             1
18      David             Pinker            13
19  Millicent            

_**Q12**_: Find the facilities with their usage by member, but not guests

In [14]:
query_q12 = '''
SELECT f.name AS facility_name,
        COUNT(b.memid) AS usage_count
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
WHERE b.memid != 0 -- Exclude guests
GROUP BY f.name;
'''

In [16]:
df_q12 = pd.read_sql_query(query_q12, conn)
print("\nFacilities usage by member:")
print(df_q12)


Facilities usage by member:
     facility_name  usage_count
0  Badminton Court          344
1   Massage Room 1          421
2   Massage Room 2           27
3       Pool Table          783
4    Snooker Table          421
5     Squash Court          195
6     Table Tennis          385
7   Tennis Court 1          308
8   Tennis Court 2          276


_**Q13**_: Find the facilities usage by month, but not guests

In [17]:
query_q13 = '''
SELECT f.name AS facility_name, 
       strftime('%Y-%m', b.starttime) AS month, 
       COUNT(b.memid) AS usage_count
FROM Facilities f
JOIN Bookings b ON f.facid = b.facid
WHERE b.memid != 0  -- Exclude guests
GROUP BY f.name, month
ORDER BY month;
'''

In [19]:
df_q13 = pd.read_sql_query(query_q13, conn)
print("\nFacilities usage by month:")
print(df_q13)


Facilities usage by month:
      facility_name    month  usage_count
0   Badminton Court  2012-07           51
1    Massage Room 1  2012-07           77
2    Massage Room 2  2012-07            4
3        Pool Table  2012-07          103
4     Snooker Table  2012-07           68
5      Squash Court  2012-07           23
6      Table Tennis  2012-07           48
7    Tennis Court 1  2012-07           65
8    Tennis Court 2  2012-07           41
9   Badminton Court  2012-08          132
10   Massage Room 1  2012-08          153
11   Massage Room 2  2012-08            9
12       Pool Table  2012-08          272
13    Snooker Table  2012-08          154
14     Squash Court  2012-08           85
15     Table Tennis  2012-08          143
16   Tennis Court 1  2012-08          111
17   Tennis Court 2  2012-08          109
18  Badminton Court  2012-09          161
19   Massage Room 1  2012-09          191
20   Massage Room 2  2012-09           14
21       Pool Table  2012-09          408
22    

In [20]:
conn.close()